# Model
Create neural network using Keras & TensorFlow

## Import Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import backend as K

Using TensorFlow backend.


In [2]:
def smooth_L1_loss(y_true, y_pred):
    THRESHOLD = K.variable(1.0)
    mae = K.abs(y_true - y_pred)
    flag = K.greater(mae, THRESHOLD)
    loss = K.mean(K.switch(flag, (mae - 0.5), K.pow(mae, 2)), axis=-1)
    return loss

In [3]:
X_train = pd.read_csv('processed data/predictors_train.csv', index_col = 0)
y_train = pd.read_csv('processed data/target_train.csv', names = ['price'])

X_test = pd.read_csv('processed data/predictors_test.csv', index_col = 0)
y_test = pd.read_csv('processed data/target_test.csv', names = ['price'])

In [4]:
fft_cols = ['fft 3', 'fft 6', 'fft 9']  # list of fourier transform column names

#re-create entire microsoft adjusted closing price DataFrame
y = y_train.append(y_test)

#re-create fourier transforms as np.complex128 (instead of converting existing columns)
close_fft = np.fft.fft(np.asarray(y['price'].tolist()))
fft_df = pd.DataFrame({'fft':close_fft})
fft_list = np.asarray(fft_df['fft'].tolist())
for num_ in [3, 6, 9]:
    fft_list_m10= np.copy(fft_list); fft_list_m10[num_:-num_]=0
    fft_df['fft {}'.format(num_)] = np.fft.ifft(fft_list_m10)
fft_df.drop(['fft'], axis = 1, inplace = True)
fft_df = fft_df.set_index(y.index)

for col in fft_cols:
    X_train[col] = fft_df[col].iloc[0:-501]
    X_test[col] = fft_df[col].iloc[-501:]

In [6]:
#convert testing/training sets to numpy arrays
X_train = np.array(X_train.values)
y_train = np.array(y_train.values)

X_test = np.array(X_test.values)
y_test = np.array(y_test.values)

In [7]:
#reshape data for input into the LSTM model
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [43]:
sequence_length = 40  # number of days before predicting
n_features = X_train.shape[2]  # number of features in the data set
time_step = 1  # number of samples

In [44]:
model = Sequential()
#1 layer LSTM with 500 nodes and Xavier (glorot_uniform) activation and L1 loss function
model.add(LSTM(500, batch_input_shape=(sequence_length, time_step, n_features),
                       kernel_initializer='glorot_uniform', stateful=True))
#add LSTM layer w 250 nodes here
#add dropout layer here
#add dense layer here
#add dropout layer here
model.add(Dense(1, input_shape=(500,)))  # single node dense output layer with prediction
model.compile(loss=smooth_L1_loss, optimizer='adam')

model.summary()  # show model summary

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (40, 500)                 1064000   
_________________________________________________________________
dense_6 (Dense)              (40, 1)                   501       
Total params: 1,064,501
Trainable params: 1,064,501
Non-trainable params: 0
_________________________________________________________________


In [45]:
#fit model to training data
n_epochs = 20
for i in range(n_epochs):
    model.fit(X_train, y_train, epochs=1, batch_size=40, verbose=0, shuffle=False)

In [46]:
prd = model.predict(X_test)

ValueError: In a stateful network, you should only pass inputs with a number of samples that can be divided by the batch size. Found: 501 samples. Batch size: 32.

In [ ]:
score = mean_squared_error(y_test, prd)